In [5]:
#Wczytanie dokumentu
import os  
import pandas as pd
import kagglehub

path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

csv_path = os.path.join(path, "IMDB Dataset.csv")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(df.head())
else:
    print(f"Plik {csv_path} nie istnieje. Sprawdź nazwę pliku i ścieżkę.")


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\LENOVO\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [6]:
#Funkcja czyszcząca do lematyzacji tekstu i usuwania interpunkcji, słów stop, białych znaków 
import spacy
nlp = spacy.load("en_core_web_sm")
def clean(text):
    doc = nlp(text)
    doc_cleaned = [
        token for token in doc 
        if not (token.is_punct or token.is_stop or token.is_space or token.is_bracket)
    ]
    doc_lemmatized = [token.lemma_ for token in doc_cleaned]
    text_lemmatized = " ".join(doc_lemmatized)
    
    return text_lemmatized


In [7]:
from tqdm import tqdm
tqdm.pandas(desc="Czyszczenie tekstów")  
df['review'] = df['review'].progress_apply(clean)
df.head()

Czyszczenie tekstów: 100%|██████████| 50000/50000 [31:30<00:00, 26.44it/s]  


,review,sentiment
0,reviewer mention watch 1 Oz episode hook right...,positive
1,wonderful little production br /><br />the fil...,positive
2,think wonderful way spend time hot summer week...,positive
3,basically family little boy Jake think zombie ...,negative
4,Petter Mattei love Time money visually stunnin...,positive


In [35]:
# Wektoryzacja tekstu za pomocą TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tqdm.pandas(desc="Wektoryzacja")  
text_vectorized = tfidf_vectorizer.fit_transform(df['review'])  
x = text_vectorized
y = df['sentiment']

In [22]:
# Wektoryzacja tekstu za pomocą CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
text_vectorized = count_vectorizer.fit_transform(df['review'])
x = text_vectorized
y = df['sentiment']

In [28]:
# Wektoryzacja tekstu za pomocą HashingVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
hashing_vectorizer = HashingVectorizer(n_features=5000) 
text_vectorized = hashing_vectorizer.fit_transform(df['review'])
x = text_vectorized
y = df['sentiment']

In [36]:
#Model 1 RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42,stratify=y)

# Inicjalizacja klasyfikatora Random Forest
rf_classifier = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42)

# Trenowanie modelu na zbiorze treningowym
rf_classifier.fit(x_train, y_train)

# Predykcja 
y_predicted_train = rf_classifier.predict(x_train)
y_predicted_test = rf_classifier.predict(x_test)

print("Macierz pomyłek na zbiorze treningowym:")
print(confusion_matrix(y_train, y_predicted_train))
print("Macierz pomyłek na zbiorze testowym:")
print(confusion_matrix(y_test, y_predicted_test))
print("Raport klasyfikacji na zbiorze treningowym:")
print(classification_report(y_train, y_predicted_train))
print("Raport klasyfikacji na zbiorze testowym:")
print(classification_report(y_test, y_predicted_test))

Macierz pomyłek na zbiorze treningowym:
[[10401  2099]
 [ 1054 11446]]
Macierz pomyłek na zbiorze testowym:
[[ 9913  2587]
 [ 1567 10933]]
Raport klasyfikacji na zbiorze treningowym:
              precision    recall  f1-score   support

    negative       0.91      0.83      0.87     12500
    positive       0.85      0.92      0.88     12500

    accuracy                           0.87     25000
   macro avg       0.88      0.87      0.87     25000
weighted avg       0.88      0.87      0.87     25000

Raport klasyfikacji na zbiorze testowym:
              precision    recall  f1-score   support

    negative       0.86      0.79      0.83     12500
    positive       0.81      0.87      0.84     12500

    accuracy                           0.83     25000
   macro avg       0.84      0.83      0.83     25000
weighted avg       0.84      0.83      0.83     25000



In [37]:
#Model 2 LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42, stratify=y)

# Inicjalizacja klasyfikatora Logistic Regression
lr_classifier = LogisticRegression(solver='saga', random_state=42)

# Trenowanie modelu na zbiorze treningowym
lr_classifier.fit(x_train, y_train)

# Predykcja 
y_predicted_train = lr_classifier.predict(x_train)
y_predicted_test = lr_classifier.predict(x_test)

print("Macierz pomyłek na zbiorze treningowym:")
print(confusion_matrix(y_train, y_predicted_train))
print("Macierz pomyłek na zbiorze testowym:")
print(confusion_matrix(y_test, y_predicted_test))
print("Raport klasyfikacji na zbiorze treningowym:")
print(classification_report(y_train, y_predicted_train))
print("Raport klasyfikacji na zbiorze testowym:")
print(classification_report(y_test, y_predicted_test))


Macierz pomyłek na zbiorze treningowym:
[[11525   975]
 [  746 11754]]
Macierz pomyłek na zbiorze testowym:
[[10852  1648]
 [ 1209 11291]]
Raport klasyfikacji na zbiorze treningowym:
              precision    recall  f1-score   support

    negative       0.94      0.92      0.93     12500
    positive       0.92      0.94      0.93     12500

    accuracy                           0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000

Raport klasyfikacji na zbiorze testowym:
              precision    recall  f1-score   support

    negative       0.90      0.87      0.88     12500
    positive       0.87      0.90      0.89     12500

    accuracy                           0.89     25000
   macro avg       0.89      0.89      0.89     25000
weighted avg       0.89      0.89      0.89     25000



In [21]:
#Model 3 SVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42, stratify=y)

# Inicjalizacja klasyfikatora Support Vector Machine
svm_classifier = SVC(random_state=42)

# Trenowanie modelu na zbiorze treningowym
svm_classifier.fit(x_train, y_train)

# Predykcja 
y_predicted_train = svm_classifier.predict(x_train)
y_predicted_test = svm_classifier.predict(x_test)

print("Macierz pomyłek na zbiorze treningowym:")
print(confusion_matrix(y_train, y_predicted_train))
print("Macierz pomyłek na zbiorze testowym:")
print(confusion_matrix(y_test, y_predicted_test))
print("Raport klasyfikacji na zbiorze treningowym:")
print(classification_report(y_train, y_predicted_train))
print("Raport klasyfikacji na zbiorze testowym:")
print(classification_report(y_test, y_predicted_test))


Macierz pomyłek na zbiorze treningowym:
[[12398   102]
 [  105 12395]]
Macierz pomyłek na zbiorze testowym:
[[10899  1601]
 [ 1156 11344]]
Raport klasyfikacji na zbiorze treningowym:
              precision    recall  f1-score   support

    negative       0.99      0.99      0.99     12500
    positive       0.99      0.99      0.99     12500

    accuracy                           0.99     25000
   macro avg       0.99      0.99      0.99     25000
weighted avg       0.99      0.99      0.99     25000

Raport klasyfikacji na zbiorze testowym:
              precision    recall  f1-score   support

    negative       0.90      0.87      0.89     12500
    positive       0.88      0.91      0.89     12500

    accuracy                           0.89     25000
   macro avg       0.89      0.89      0.89     25000
weighted avg       0.89      0.89      0.89     25000



In [3]:
import pickle

# Wektoryzacja tekstu za pomocą TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tqdm.pandas(desc="Wektoryzacja")  
vectorized_text = tfidf_vectorizer.fit_transform(df['review'])  
x = vectorized_text
y = df['sentiment']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42, stratify=y)

# Model 1: Logistic Regression
lr_classifier = LogisticRegression(solver='saga', random_state=42)
lr_classifier.fit(x_train, y_train)

# Model 2: Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(x_train, y_train)

# Zapisywanie wektoryzatora
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)

# Zapisywanie modelu Logistic Regression
with open("logistic_regression_model.pkl", "wb") as f:
    pickle.dump(lr_classifier, f)

# Zapisywanie modelu Random Forest
with open("random_forest_model.pkl", "wb") as f:
    pickle.dump(rf_classifier, f)

print("Modele i wektoryzator zostały zapisane.")


NameError: name 'tqdm' is not defined

In [4]:
import tkinter as tk
from tkinter import ttk
import spacy
import pickle

# Funkcja czyszcząca do lematyzacji tekstu
nlp = spacy.load("en_core_web_sm")
def clean_text(text):
    doc = nlp(text)
    doc_cleaned = [
        token for token in doc
        if not (token.is_punct or token.is_stop or token.is_space or token.is_bracket)
    ]
    doc_lemmatized = [token.lemma_ for token in doc_cleaned]
    return " ".join(doc_lemmatized)

# Wczytanie modeli i wektoryzatora
with open("tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

with open("logistic_regression_model.pkl", "rb") as f:
    logistic_regression_model = pickle.load(f)

with open("random_forest_model.pkl", "rb") as f:
    random_forest_model = pickle.load(f)

# Funkcja do klasyfikacji tekstu
def classify_text():
    user_input = input_text.get("1.0", tk.END).strip()
    if not user_input:
        result_label.config(text="Wprowadź recenzję do analizy.")
        return

    cleaned_text = clean_text(user_input)
    vectorized_text = tfidf_vectorizer.transform([cleaned_text])

    # Predykcja przy użyciu modeli
    sentiment_lr = logistic_regression_model.predict(vectorized_text)[0]
    sentiment_rf = random_forest_model.predict(vectorized_text)[0]

    # Wyświetlenie wyników
    result_label.config(
        text=f"Sentyment wg Logistic Regression: {'Pozytywny' if sentiment_lr == 'positive' else 'Negatywny'}\n"
             f"Sentyment wg Random Forest: {'Pozytywny' if sentiment_rf == 'positive' else 'Negatywny'}"
    )

# GUI aplikacji
app = tk.Tk()
app.title("Analiza Sentymentu")
app.geometry("500x400")

# Etykiety i pola tekstowe
label = tk.Label(app, text="Wprowadź recenzję:", font=("Arial", 14))
label.pack(pady=10)

input_text = tk.Text(app, height=10, width=50)
input_text.pack(pady=10)

classify_button = ttk.Button(app, text="Analizuj", command=classify_text)
classify_button.pack(pady=10)

result_label = tk.Label(app, text="", font=("Arial", 12), justify="left")
result_label.pack(pady=10)

app.mainloop()
